# Jakobian geometryczny

Na podstawie kolejnych macierzy przekształceń wyprowadza jakobian geometryczny.

In [1]:
from sympy import *
from RobotykaLib import *
q1, q2, q3, q4 = symbols('q1 q2 q3 q4', real=True)  # q
a, a1, a2, a3 = symbols('a a1 a2 a3', real=True)    # a
d, d1, d2, d3 = symbols('d d1 d2 d3', real=True)    # d
l, l1, l2, l3 = symbols('l l1 l2 l3', real=True)
alfa, beta,gamma = symbols('\\alpha \\beta \\gamma', real=True)
x, y, z = symbols('x y z')                          # osie

## Wprowadź dane: Macierz kinematyki:

# $$A_{0}^{1}=$$

In [2]:
A01 = tr(z,q1)*rot(x,-pi/2)
A01

Matrix([
[1,  0, 0,  0],
[0,  0, 1,  0],
[0, -1, 0, q1],
[0,  0, 0,  1]])

# $$A_{1}^{2}=$$

In [3]:
A12 = rot(z,q2)*tr(z,d2)*rot(x,pi/2)
A12

Matrix([
[cos(q2), 0,  sin(q2),  0],
[sin(q2), 0, -cos(q2),  0],
[      0, 1,        0, d2],
[      0, 0,        0,  1]])

# $$A_{2}^{3}=$$

In [4]:
A23 = rot(z,q3)*tr(x,a3)
A23

Matrix([
[cos(q3), -sin(q3), 0, a3*cos(q3)],
[sin(q3),  cos(q3), 0, a3*sin(q3)],
[      0,        0, 1,          0],
[      0,        0, 0,          1]])

# $$A_{0}^{2}=$$

In [5]:
A02 = A01*A12
A02

Matrix([
[ cos(q2), 0, sin(q2),  0],
[       0, 1,       0, d2],
[-sin(q2), 0, cos(q2), q1],
[       0, 0,       0,  1]])

# $$A_{0}^{3}=K=$$

In [6]:
K = A02*A23
K
# K = [[-sin(q1)*cos(q3), -cos(q1), sin(q1)*sin(q3), -sin(q1)*(a3*cos(q3)+a3) + q2*cos(q1)],
# [cos(q1)*cos(q3), -sin(q1), -cos(q1)*sin(q3), cos(q1)*(a3*cos(q3)+a2)+ q2*(sin(q1))],
# [sin(q3), 0, cos(q3), d1 + a3 * sin(q3)],
# [0,0,0,1]]

Matrix([
[ cos(q2)*cos(q3), -sin(q3)*cos(q2), sin(q2),       a3*cos(q2)*cos(q3)],
[         sin(q3),          cos(q3),       0,          a3*sin(q3) + d2],
[-sin(q2)*cos(q3),  sin(q2)*sin(q3), cos(q2), -a3*sin(q2)*cos(q3) + q1],
[               0,                0,       0,                        1]])

In [7]:
K = Matrix(K)
K

Matrix([
[ cos(q2)*cos(q3), -sin(q3)*cos(q2), sin(q2),       a3*cos(q2)*cos(q3)],
[         sin(q3),          cos(q3),       0,          a3*sin(q3) + d2],
[-sin(q2)*cos(q3),  sin(q2)*sin(q3), cos(q2), -a3*sin(q2)*cos(q3) + q1],
[               0,                0,       0,                        1]])

### Macierz rotacji
# $$R=$$

In [8]:
R = K.minor_submatrix(3,3)
R

Matrix([
[ cos(q2)*cos(q3), -sin(q3)*cos(q2), sin(q2)],
[         sin(q3),          cos(q3),       0],
[-sin(q2)*cos(q3),  sin(q2)*sin(q3), cos(q2)]])

### Wektor transpozycji
# $$T=$$

In [9]:
T = K.copy()
T.row_del(3)
for i in range(0,3):
    T.col_del(0)
T

Matrix([
[      a3*cos(q2)*cos(q3)],
[         a3*sin(q3) + d2],
[-a3*sin(q2)*cos(q3) + q1]])

# $$q=$$

In [10]:
q = [q1, q2, q3]
Matrix(q)

Matrix([
[q1],
[q2],
[q3]])

### Góra jakobianu manipulatora

In [11]:
dT = []
for i in range(0,3):
    row = []
    for qi in q:
        row.append(T[i].diff(qi).doit())
    dT.append(row)

Matrix(dT)


Matrix([
[0, -a3*sin(q2)*cos(q3), -a3*sin(q3)*cos(q2)],
[0,                   0,          a3*cos(q3)],
[1, -a3*cos(q2)*cos(q3),  a3*sin(q2)*sin(q3)]])

In [12]:
def to_vector(A):
    v = []
    v.append(A[2,1])
    v.append(A[0,2])
    v.append(A[1,0])
    return v

### Dół jakobianu manipulatora

In [13]:
J_dol = []
for i in range(0,3):
    dR = R.diff(q[i]).doit()
    print(f"--- dR/dq_{i+1} R^T ---")
    display(dR)
    dR = simplify(dR * R.transpose())
    display(dR)
    v = to_vector(dR)
    J_dol.append(v)
    display(Matrix(v))

--- dR/dq_1 R^T ---


Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

Matrix([
[0],
[0],
[0]])

--- dR/dq_2 R^T ---


...

Matrix([
[0],
[1],
[0]])

--- dR/dq_3 R^T ---


Matrix([
[-sin(q3)*cos(q2), -cos(q2)*cos(q3), 0],
[         cos(q3),         -sin(q3), 0],
[ sin(q2)*sin(q3),  sin(q2)*cos(q3), 0]])

Matrix([
[      0, -cos(q2),        0],
[cos(q2),        0, -sin(q2)],
[      0,  sin(q2),        0]])

Matrix([
[sin(q2)],
[      0],
[cos(q2)]])

Matrix([
[0, 0, cos(q1)],
[0, 0, sin(q1)],
[1, 0,       0]])

### Jakobian manipulatora
# $$J=$$

In [16]:
J = Matrix(dT).col_join(Matrix(J_dol).transpose())
J

Matrix([
[0,  a2*cos(q2)*cos(q3), -a2*sin(q2)*sin(q3)],
[0,                   0,          a2*cos(q3)],
[1, -a2*sin(q2)*cos(q3), -a2*sin(q3)*cos(q2)],
[0,                   0,            -cos(q2)],
[0,                   1,                   0],
[0,                   0,             sin(q2)]])